In [2]:
# Libraries

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import uuid
import re

In [3]:
# define classes of information to be collected

class Reviews:
    def __init__(self, review_id, item_id, review_time, review_variant, star_rating, review_content):
        self.review_id = review_id
        self.item_id=item_id
        self.review_time = review_time
        self.review_variant = review_variant
        self.star_rating  = star_rating
        self.review_content = review_content

class Items:
    def __init__(self, item_id, item_name, item_ratings, ratings_num, item_sold, shop_name, item_price, item_url, item_desc, image_url):
        self.item_id=item_id
        self.item_name=item_name
        self.item_ratings=item_ratings
        self.ratings_num=ratings_num
        self.item_sold=item_sold
        self.shop_name=shop_name
        self.item_price=item_price
        self.item_url=item_url
        self.item_desc=item_desc
        self.image_url=image_url


In [4]:
#Create Chrome options
chrome_options= Options()
chrome_options.add_argument('--no sandbox')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('disable-inforbars')

# Set the path to the Chrome driver executable
chrome_driver_path = 'D:\Chrome web driver\chromedriver-win64\chromedriver.exe'

# Create an instance of the Chrome browser
browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)

# Navigate to a URL 
browser.get("https://shopee.com.my/")

C:\Users\Asus\AppData\Local\Temp\ipykernel_5436\1499064468.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)


In [5]:
soup=BeautifulSoup(browser.page_source, "html.parser")

the_links=[]

parent_div = soup.findAll('div', class_="col-xs-2-4 shopee-search-item-result__item")

for parent_div in parent_div:
    a=parent_div.find('a')
    if (a!=None):
        href=a.get('href')
        the_links.append(href)

        # print the links
        print(href)
        

/🔥CRAZY-SALE🔥-Upgrade-Japanese-slipper-Comfortable-Sole-Shower-Slippers-House-Slippers-Indoor-Slipper-Home-Sandas-i.12609016.15177535273?sp_atk=60e5659a-befd-4a08-a30d-2b75c41d294a&xptdk=60e5659a-befd-4a08-a30d-2b75c41d294a
/Women-Heels-Sandals-Simple-Fashion-Atmosphere-Comfortable-Non-slip-Ladies-Heel-Sandals-i.142617586.22035371739?sp_atk=e0c429a2-cbef-4d35-8ac6-b0bcb2b89ecf&xptdk=e0c429a2-cbef-4d35-8ac6-b0bcb2b89ecf
/Shopee-Warehouse-Ready-Stock-Ladies-White-Leather-Shoes-Pearls-Mary-Jane-Shoes-i.78992831.20712924226?sp_atk=d754cce3-80dd-4799-b61a-4d39ddf53070&xptdk=d754cce3-80dd-4799-b61a-4d39ddf53070
/HEUS-Milita-High-Platform-Boots-(Ready-Stock)-i.17071880.15333809147?sp_atk=e32630ea-e74b-4326-8f07-538f031b2af9&xptdk=e32630ea-e74b-4326-8f07-538f031b2af9
/Kasut-Perempuan-Wanita-Selipar-Sandal-Shoes-Women-Ladies-女生鞋-i.416060590.15085971297?sp_atk=e834ea1f-7888-4367-a484-f56257118e20&xptdk=e834ea1f-7888-4367-a484-f56257118e20
/【888-Shoes-Ready-Stock】high-heeled-sandals-fashion-Point

In [6]:
len(the_links)

60

In [7]:
items_list=[]
reviews_list=[]

In [8]:
for i in range (0,len(the_links)+1):
    
    # open link
    links='https://shopee.com.my'+ the_links[i]
    browser.get(links)
    time.sleep(10)

    # parse html using beautifulsoup
    soup=BeautifulSoup(browser.page_source, "html.parser")

    # extract product information
    id=uuid.uuid4()
    title = soup.find('div', attrs={'class':'_44qnta'}).find('span').get_text(strip=True)
    ratings=soup.find('div', class_="_1k47d8 _046PXf").get_text(strip=True)
    ratings_num=soup.select('div._1k47d8:not(div._1k47d8._046PXf)')[0].get_text(strip=True)
    sold=soup.find('div', class_="e9sAa2").get_text(strip=True)
    shop_name=soup.find('div',class_="VlDReK").get_text(strip=True)
    price = soup.find('div', class_='pqTWkA').get_text(strip=True)

    # extract description
    descriptions = soup.find_all('p', class_='irIKAp')
    

    desc_array=[]
    for description in descriptions:
        desc = re.sub(r'<[^>]+>', '', str(description))  # Remove HTML tags
        desc = desc.strip()  # Remove leading/trailing whitespaces
        desc_array.append(desc)

    text = "\n".join(desc_array)

    # extract picture url
    img_tag= soup.find('img', class_='_7D4JtJ')

    if img_tag:
        image = img_tag['src']
    else:
        image=None

    # append item to a list
    items_list.append(Items(id,title,ratings,ratings_num,sold,shop_name, price, links, description, image))

    # reviews extraction
    elements = browser.find_elements(By.CLASS_NAME,"product-rating-overview__filter")
    counter=5
    for j in range(1,6):
        # select star categories
        elements[j].click()
        
        # loop is for each category
        for k in range (0,31): 
            soup=BeautifulSoup(browser.page_source, "html.parser")
            review_containers=soup.findAll('div',class_="shopee-product-rating__main")

            no_reviews_found = False

            for rc in review_containers:
                review_id=uuid.uuid4()
                item_id=id
                time_var=rc.find('div',class_="shopee-product-rating__time").get_text(strip=True).split("|")

                if(len(time_var)==2):
                    review_time=time_var[0]
                    variant=time_var[1]
                else:
                    review_time=time_var[0]
                    variant="None"

                star_rating=counter
                reviews=rc.find('div',class_="Rk6V+3")

                if(reviews!=None):
                    reviews=reviews.get_text(strip=True)
                else:
                    reviews="None"
                    no_reviews_found = True

                reviews_list.append(Reviews(review_id, item_id, review_time,variant,star_rating,reviews))
            
            # no more reviews break
            if no_reviews_found:
                print("No reviews found, breaking the loop")
                break

            time.sleep(1)
            try:
                WebDriverWait(browser, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class^='shopee-icon-button shopee-icon-button--right "))).click()
            except:
                print("not found")
                break
            time.sleep(1)
        
        counter=counter-1
        time.sleep(2)
    print(i) 
    
    

No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
0
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
1
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
2
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
3
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
4
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, 

IndexError: list index out of range

In [9]:
items_df = pd.DataFrame([vars(obj) for obj in items_list])
reviews_df = pd.DataFrame([vars(obj) for obj in reviews_list])

In [10]:
items_df.shape

(60, 10)

In [14]:
items_df.head()

,item_id,item_name,item_ratings,ratings_num,item_sold,shop_name,item_price,item_url,item_desc,image_url
0,2285b2b6-61ae-4596-8233-6dc65b7018d7,🔥CRAZY SALE🔥 Upgrade Japanese slipper Comforta...,4.7,8.3k,31.2k,allblue5367,RM3.80 - RM5.00,https://shopee.com.my/🔥CRAZY-SALE🔥-Upgrade-Jap...,[Product description:\n📌Product name: ✨2021 Up...,https://down-my.img.susercontent.com/file/my-1...
1,47c01356-4e22-4001-bc36-d95023d486dc,Women Heels Sandals Simple Fashion Atmosphere ...,4.9,521,1.4k,Max Queen Fashion Store,RM34.12,https://shopee.com.my/Women-Heels-Sandals-Simp...,[Size: 35 36 37 38 39 40 41 42],https://down-my.img.susercontent.com/file/sg-1...
2,a0963a88-5578-4d65-8302-374198162721,Shopee Warehouse Ready Stock Ladies White Leat...,4.8,1.3k,3.4k,Popyoung Store,RM19.90 - RM25.80,https://shopee.com.my/Shopee-Warehouse-Ready-S...,[Due to the resolution in the resolution of th...,https://down-my.img.susercontent.com/file/sg-1...
3,4f560d0e-7ea6-4169-8248-82cb82832938,HEUS Milita High Platform Boots (Ready Stock),4.9,3.9k,10.3k,HEUS Shoes Wholesale,RM22.68 - RM23.90,https://shopee.com.my/HEUS-Milita-High-Platfor...,[#sale #korean #kasut #shoes #heels #womanshoe...,https://down-my.img.susercontent.com/file/92ef...
4,3485bfb8-c2e3-4963-a195-61f4e1b92ce6,Kasut Perempuan Wanita Selipar Sandal Shoes Wo...,4.7,724,2.5k,LK luxury house,RM7.09 - RM7.48,https://shopee.com.my/Kasut-Perempuan-Wanita-S...,[📦READY STOCK\n✅DROPSHIP | RESELLER | WHOLESAL...,https://down-my.img.susercontent.com/file/a626...


In [11]:
reviews_df.shape

(5306, 6)

In [13]:
reviews_df.tail()

,review_id,item_id,review_time,review_variant,star_rating,review_content
5301,e79597cc-4d2a-42a6-b70d-496991fe52f8,27d8f34b-cf85-4cfb-93bc-3f6a06c16e3a,2023-04-20 22:13,"Variation: Black,40",1,"Mmg taksesuai kalau nak beli , takselasa langs..."
5302,b1c7c625-83d9-4c11-b226-f264c25d78c9,27d8f34b-cf85-4cfb-93bc-3f6a06c16e3a,2023-03-08 10:42,"Variation: Black,37",1,Comfortability:blm cubaTerlalu besar untuk sai...
5303,7e276d99-0187-4337-8d7d-548f75c74bda,27d8f34b-cf85-4cfb-93bc-3f6a06c16e3a,2023-02-19 15:19,"Variation: khakis,37",1,None
5304,33da1218-4d62-4d5e-87e8-1be5fb185fbf,27d8f34b-cf85-4cfb-93bc-3f6a06c16e3a,2023-03-02 22:20,"Variation: Black,38",1,Material:okPenghataran agak slow
5305,aa8eeb2d-8bde-4d4e-91fb-0eb50fe114d8,27d8f34b-cf85-4cfb-93bc-3f6a06c16e3a,2023-06-22 17:29,"Variation: Black,36",1,None


In [16]:
reviews_df.to_json('womenshoes_reviews_1.json', orient='records', lines=True, force_ascii=False, default_handler=str)
items_df.to_json('womenshoes_items_1.json', orient='records', lines=True, force_ascii=False, default_handler=str)